# Best Estimate

The best estimate I can do with current information, making use of meter readings collected over the past year, as well as some climate data on hours of sunshine from the Met Office, and some basic assumptions about costs, unit rates and solar panel outputs.

In [67]:
import json
import pandas as pd
import yaml

In [68]:
with open("../data/ballpark_figures.yaml", "r", encoding="utf-8") as filepath:
    parameters = yaml.load(filepath, Loader=yaml.FullLoader)

ballpark_installation_cost = parameters["installation_cost"]
ballpark_sell_back_rate = parameters["sell_back_rate"]
expected_unit_rate = parameters["expected_unit_rate"]
solar_panel_total_output = parameters["solar_panel_total_output"]
solar_panel_lifetime = parameters["solar_panel_lifetime"]

print(json.dumps(parameters, indent=4))

{
    "installation_cost": 17850,
    "expected_unit_rate": 0.2839,
    "sell_back_rate": 0.1422,
    "yearly_bill": 14126.03,
    "ave_hours_daylight": 3.44,
    "solar_panel_total_output": 12.75,
    "solar_panel_lifetime": 25
}


In [69]:
# Meter readings collected from May 2023 - May 2024:
meter_readings_df = pd.read_excel(
    "~/Documents/committee/meter_readings.xlsx", sheet_name="meter_readings", usecols=[0, 1, 2]
)

# Data on hours of sunshine gathered from Met Office (https://www.metoffice.gov.uk/research/climate/maps-and-data/uk-climate-averages/gcey2u2yw)
hours_of_sunshine_df = pd.read_excel("~/Documents/committee/meter_readings.xlsx", sheet_name="hours_of_sunshine")

In [70]:
meter_readings_df.head()

,date,meter_reading,weekly_usage
0,2023-04-30,59059.0,NaN
1,2023-05-07,59127.0,68.0
2,2023-05-14,59225.0,98.0
3,2023-05-21,59353.0,128.0
4,2023-05-28,59467.0,114.0


In [71]:
hours_of_sunshine_df

,month,hours_of_sunshine,daily_average
0,January,42.70,1.377419
1,February,66.93,2.369204
2,March,101.15,3.262903
3,April,148.19,4.939667
4,May,183.30,5.912903
5,June,150.13,5.004333
6,July,136.14,4.391613
7,August,136.15,4.391935
8,September,112.91,3.763667
9,October,85.41,2.755161


In [72]:
meter_readings_df["month"] = meter_readings_df["date"].apply(lambda x: x.month_name())

monthly_usage_df = meter_readings_df.groupby("month", as_index=False)["weekly_usage"].sum()
monthly_usage_df.rename({"weekly_usage": "monthly_usage"}, axis=1, inplace=True)
monthly_usage_df["average_daily_usage"] = monthly_usage_df["monthly_usage"] / [31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [73]:
# Multiplying solar panel output by hours of sunshine gives output for month:
hours_of_sunshine_df["solar_panel_output"] = hours_of_sunshine_df.hours_of_sunshine.apply(lambda x: x * solar_panel_total_output)

# Divide by number of days to get daily average:
hours_of_sunshine_df["average_daily_output"] = hours_of_sunshine_df["solar_panel_output"] / [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

hours_of_sunshine_df

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output
0,January,42.70,1.377419,544.4250,17.562097
1,February,66.93,2.369204,853.3575,30.477054
2,March,101.15,3.262903,1289.6625,41.602016
3,April,148.19,4.939667,1889.4225,62.980750
4,May,183.30,5.912903,2337.0750,75.389516
5,June,150.13,5.004333,1914.1575,63.805250
6,July,136.14,4.391613,1735.7850,55.993065
7,August,136.15,4.391935,1735.9125,55.997177
8,September,112.91,3.763667,1439.6025,47.986750
9,October,85.41,2.755161,1088.9775,35.128306


In [74]:
# By merging datasets on month, we can see how usage compares with generation:
output_and_usage = pd.merge(hours_of_sunshine_df, monthly_usage_df, how="inner", on="month")

We can see the figures for usage and generation by month below. As a side-note, the usage (shown in `average_daily_usage`), particularly during the summer months, seems very high to me, so I'd be curious to know what others think.

In [75]:
output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage
0,January,42.70,1.377419,544.4250,17.562097,743.0,23.967742
1,February,66.93,2.369204,853.3575,30.477054,782.0,26.066667
2,March,101.15,3.262903,1289.6625,41.602016,945.0,30.483871
3,April,148.19,4.939667,1889.4225,62.980750,686.0,22.129032
4,May,183.30,5.912903,2337.0750,75.389516,580.0,19.333333
5,June,150.13,5.004333,1914.1575,63.805250,379.0,12.225806
6,July,136.14,4.391613,1735.7850,55.993065,450.0,15.000000
7,August,136.15,4.391935,1735.9125,55.997177,483.0,17.097345
8,September,112.91,3.763667,1439.6025,47.986750,564.0,18.193548
9,October,85.41,2.755161,1088.9775,35.128306,917.0,30.566667


Let's assume now that if we install a 10kWh battery, then the first 10kWh generated will be stored in the battery to be used, and any electricity generated above this will be sold to the grid. 

This is an approximation, and may not be totally accurate; it's an attempt to account for the predominant use in the evenings so that if we generate 10kWh by 3pm for example, no more electricity generated after that can be kept until the battery starts to be depleted. During the summer time I imagine most use is during Sunday morning, so this would not hold then, but during the week if the halls were not in use, then it would be valid.

In [76]:
output_and_usage["average_daily_units_to_sell"] = output_and_usage["average_daily_output"].apply(lambda x: max(0, x - 10))
output_and_usage["average_daily_units_to_use"] = output_and_usage["average_daily_output"].apply(lambda x: min(x, 10))

# Converting back to monthly:
output_and_usage["average_monthly_units_to_sell"] = output_and_usage["average_daily_units_to_sell"] * [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
output_and_usage["average_monthly_units_to_use"] = output_and_usage["average_daily_units_to_use"] * [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage,average_daily_units_to_sell,average_daily_units_to_use,average_monthly_units_to_sell,average_monthly_units_to_use
0,January,42.70,1.377419,544.4250,17.562097,743.0,23.967742,7.562097,10,234.4250,310
1,February,66.93,2.369204,853.3575,30.477054,782.0,26.066667,20.477054,10,573.3575,280
2,March,101.15,3.262903,1289.6625,41.602016,945.0,30.483871,31.602016,10,979.6625,310
3,April,148.19,4.939667,1889.4225,62.980750,686.0,22.129032,52.980750,10,1589.4225,300
4,May,183.30,5.912903,2337.0750,75.389516,580.0,19.333333,65.389516,10,2027.0750,310
5,June,150.13,5.004333,1914.1575,63.805250,379.0,12.225806,53.805250,10,1614.1575,300
6,July,136.14,4.391613,1735.7850,55.993065,450.0,15.000000,45.993065,10,1425.7850,310
7,August,136.15,4.391935,1735.9125,55.997177,483.0,17.097345,45.997177,10,1425.9125,310
8,September,112.91,3.763667,1439.6025,47.986750,564.0,18.193548,37.986750,10,1139.6025,300
9,October,85.41,2.755161,1088.9775,35.128306,917.0,30.566667,25.128306,10,778.9775,310


In [77]:
def get_expected_saving(
        monthly_units_to_sell,
        monthly_units_to_use,
        monthly_usage,
        unit_rate=expected_unit_rate,
        sell_back_rate=ballpark_sell_back_rate
    ):
    """
    Return expected saving given stats for monthly output, monthly usage,
    expected unit rate and sell back rate.

    We take account of battery capacity so that the savings are only based on the units we would
    have available to use, plus any money gained from selling leftover units back to the grid.
    """

    # doesn't appear to be the case, but if we were to use less than what is stored in the battery,
    # we'd sell that as well:
    if monthly_usage < monthly_units_to_use:
        monthly_units_to_sell += (monthly_units_to_use - monthly_usage)
        return (monthly_usage * unit_rate) + (monthly_units_to_sell * sell_back_rate)
    return (monthly_units_to_use * unit_rate) + (monthly_units_to_sell * sell_back_rate)

output_and_usage[f"expected_saving_{expected_unit_rate}"] = output_and_usage.apply(
    lambda x: get_expected_saving(
        x["average_monthly_units_to_sell"],
        x["average_monthly_units_to_use"],
        x["monthly_usage"],
    ),
    axis=1
)

In [78]:
output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage,average_daily_units_to_sell,average_daily_units_to_use,average_monthly_units_to_sell,average_monthly_units_to_use,expected_saving_0.2839
0,January,42.70,1.377419,544.4250,17.562097,743.0,23.967742,7.562097,10,234.4250,310,121.344235
1,February,66.93,2.369204,853.3575,30.477054,782.0,26.066667,20.477054,10,573.3575,280,161.023437
2,March,101.15,3.262903,1289.6625,41.602016,945.0,30.483871,31.602016,10,979.6625,310,227.317008
3,April,148.19,4.939667,1889.4225,62.980750,686.0,22.129032,52.980750,10,1589.4225,300,311.185879
4,May,183.30,5.912903,2337.0750,75.389516,580.0,19.333333,65.389516,10,2027.0750,310,376.259065
5,June,150.13,5.004333,1914.1575,63.805250,379.0,12.225806,53.805250,10,1614.1575,300,314.703196
6,July,136.14,4.391613,1735.7850,55.993065,450.0,15.000000,45.993065,10,1425.7850,310,290.755627
7,August,136.15,4.391935,1735.9125,55.997177,483.0,17.097345,45.997177,10,1425.9125,310,290.773757
8,September,112.91,3.763667,1439.6025,47.986750,564.0,18.193548,37.986750,10,1139.6025,300,247.221475
9,October,85.41,2.755161,1088.9775,35.128306,917.0,30.566667,25.128306,10,778.9775,310,198.779600


If we assume a unit rate of £0.2839, and an installation cost of ~ £30,000 (accounting for ~ £10,000 for panel installation, and two payments of £10,000 each for an initial 10kWh battery and a replacement after ~10-15 years), then we can calculate a rough estimate of annual savings and a breakeven point:

In [79]:
estimated_annual_saving = round(output_and_usage['expected_saving_0.2839'].sum(), 2)

print(f"Estimated annual saving assuming a unit rate of £{expected_unit_rate}: £{estimated_annual_saving}")
print(
    "Estimated time until breakeven assuming total installation cost of £17,850: "
    f"{round(ballpark_installation_cost / estimated_annual_saving, 2)} years."
)

Estimated annual saving assuming a unit rate of £0.2839: £2793.6
Estimated time until breakeven assuming total installation cost of £17,850: 6.39 years.


#### Other things to note:

- Solar panel maintenance: we may need to spend £100-200 per year to clean and service panels for example.
- Efficiency degradation: according to [this link](https://www.ecowatch.com/solar/solar-panel-efficiency-over-time#:~:text=The%20average%20degradation%20is%202.5,and%2085.5%25%20after%2025%20years.), efficiency decreases by 0.5% per year. If for now we assume that annual savings are directly proportional to panel output, (a rough approximation), then we can update our calculations (see below). This shouldn't make a big difference to the final conclusion.

In [80]:
savings_by_year = []
outstanding_cost_by_year = []

outstanding_cost = ballpark_installation_cost
for year in range(1, solar_panel_lifetime + 1):
    savings = estimated_annual_saving * 0.995**year
    savings_by_year.append(estimated_annual_saving * 0.995**year)
    outstanding_cost -= savings
    outstanding_cost_by_year.append(outstanding_cost)

pd.DataFrame(
    {
        "year": list(range(1, solar_panel_lifetime + 1)),
        "annual_savings": savings_by_year,
        "outstanding_cost": outstanding_cost_by_year
    }
)


,year,annual_savings,outstanding_cost
0,1,2779.632000,15070.368000
1,2,2765.733840,12304.634160
2,3,2751.905171,9552.728989
3,4,2738.145645,6814.583344
4,5,2724.454917,4090.128428
5,6,2710.832642,1379.295785
6,7,2697.278479,-1317.982694
7,8,2683.792087,-4001.774780
8,9,2670.373126,-6672.147906
9,10,2657.021260,-9329.169167
